In [1]:
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener 
from tweepy import OAuthHandler
import json

In [5]:
consumer_key = "Jj52ijmLikNTLD3qRXSyFo2bW"
consumer_secret = "XQ3VyYt0LG5W4sotm4LfpvA99dd0KCuiD9tSZVf62Lsb02SSIJ"
access_token = "1038621976934281216-y2s3U70cy3WlUWXjx4U2uUJbfaw6LD"
access_secret = "zthp70cGjzb7VVK7cfArUuYZz7OG5KxwgJ9SG2tixhl9s"
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [13]:
@classmethod
def parse(cls, api, raw):
    status = cls.first_parse(api, raw)
    setattr(status, 'json', json.dumps(raw))
    return status
 
# Status() is the data model for a tweet
tweepy.models.Status.first_parse = tweepy.models.Status.parse
tweepy.models.Status.parse = parse
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('twitterdata.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True 
#Set the hashtag to be searched
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#covid19'],languages=["en"])

ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.

In [23]:
import json
# Tweets are stored in in file "fname". In the file used for this script, 
# each tweet was stored on one line
fname = 'twitterdata.json'
with open(fname, 'r') as f:
    
    #Create dictionary to later be stored as JSON. All data will be included
    # in the list 'data'
    users_with_geodata = {
        "data": []
    }
    all_users = []
    total_tweets = 0
    geo_tweets  = 0
    for line in f:
        tweet = json.loads(line)
        if tweet['user']['id']:
            total_tweets += 1 
            user_id = tweet['user']['id']
            if user_id not in all_users:
                all_users.append(user_id)
                
                #Give users some data to find them by. User_id listed separately 
                # to make iterating this data later easier
                user_data = {
                    "user_id" : tweet['user']['id'],
                    "tweet_txt": tweet['text'],
                    "name" : tweet['user']['name'],
                    "id": tweet['user']['id'],
                    "screen_name": tweet['user']['screen_name'],
                    "tweets" : 1,
                    "location": tweet['user']['location'],
                    "date-time": tweet['user']['created_at']
                }
                #Iterate through different types of geodata to get the variable primary_geo
#                 if tweet['coordinates']:
#                     user_data["features"]["primary_geo"] = str(tweet['coordinates'][tweet['coordinates'].keys()[1]][1]) + ", " + str(tweet['coordinates'][tweet['coordinates'].keys()[1]][0])
#                     user_data["features"]["geo_type"] = "Tweet coordinates"
#                 elif tweet['place']:
#                     user_data["features"]["primary_geo"] = tweet['place']['full_name'] + ", " + tweet['place']['country']
#                     user_data["features"]["geo_type"] = "Tweet place"
#                 else:
#                     user_data["features"]["primary_geo"] = tweet['user']['location']
#                     user_data["features"]["geo_type"] = "User location"
                #Add only tweets with some geo data to .json. Comment this if you want to include all tweets.
                if user_data["location"]:
                    users_with_geodata['data'].append(user_data)
                    geo_tweets += 1
            
            #If user already listed, increase their tweet count
            elif user_id in all_users:
                for user in users_with_geodata["data"]:
                    if user_id == user["user_id"]:
                        user["tweets"] += 1
    
    #Count the total amount of tweets for those users that had geodata            
    for user in users_with_geodata["data"]:
        geo_tweets = geo_tweets + user["tweets"]
    #Get some aggregated numbers on the data
    print ("The file included ",str(len(all_users))," unique users who tweeted with or without geo data")
    print ("The file included ",str(len(users_with_geodata['data']))," unique users who tweeted with geo data, including 'location'")
    print ("The users with geo data tweeted ",str(geo_tweets)," out of the total " ,str(total_tweets)," of tweets.")
# Save data to JSON file
with open('tweet_users_geo.json', 'w') as fout:
    fout.write(json.dumps(users_with_geodata, indent=4))

The file included  82439  unique users who tweeted with or without geo data
The file included  57921  unique users who tweeted with geo data, including 'location'
The users with geo data tweeted  132424  out of the total  106331  of tweets.


In [24]:

import csv 
  
  
# Opening JSON file and loading the data 
# into the variable data 
with open('tweet_users_geo.json') as json_file: 
    data = json.load(json_file) 
  
tweet_data = data['data'] 
  
# now we will open a file for writing 
data_file = open('data_file.csv', 'w') 
  
# create the csv writer object 
csv_writer = csv.writer(data_file) 
  
# Counter variable used for writing  
# headers to the CSV file 
count = 0
  
for tweet in tweet_data: 
    if count == 0: 
  
        # Writing headers of CSV file 
        header = tweet.keys() 
        csv_writer.writerow(header) 
        count += 1
  
    # Writing data of CSV file 
    csv_writer.writerow(tweet.values()) 
  
data_file.close() 